This code is taken from ch4 of https://github.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/

Lets start by downloading some of the files.

In [1]:
!wget https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/master/Chapter04/deepfake/deepfake_face_swap.py

--2021-05-13 02:29:04--  https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/master/Chapter04/deepfake/deepfake_face_swap.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7460 (7.3K) [text/plain]
Saving to: ‘deepfake_face_swap.py’

deepfake_face_swap. 100%[===================>]   7.29K  --.-KB/s    in 0s      

2021-05-13 02:29:04 (93.9 MB/s) - ‘deepfake_face_swap.py’ saved [7460/7460]



In [2]:
!wget https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/master/Chapter04/deepfake/deepfake_face_detection.py

--2021-05-13 02:29:20--  https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/master/Chapter04/deepfake/deepfake_face_detection.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1793 (1.8K) [text/plain]
Saving to: ‘deepfake_face_detection.py’

deepfake_face_detec 100%[===================>]   1.75K  --.-KB/s    in 0s      

2021-05-13 02:29:20 (35.1 MB/s) - ‘deepfake_face_detection.py’ saved [1793/1793]



In [3]:
!wget https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/master/Chapter04/deepfake/deepfake_config.py

--2021-05-13 02:29:56--  https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/master/Chapter04/deepfake/deepfake_config.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 415 [text/plain]
Saving to: ‘deepfake_config.py’

deepfake_config.py  100%[===================>]     415  --.-KB/s    in 0s      

2021-05-13 02:29:56 (31.8 MB/s) - ‘deepfake_config.py’ saved [415/415]



In [4]:
!wget https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/master/Chapter04/deepfake/deepfake_face_points_detection.py

--2021-05-13 02:30:37--  https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/master/Chapter04/deepfake/deepfake_face_points_detection.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 587 [text/plain]
Saving to: ‘deepfake_face_points_detection.py’

deepfake_face_point 100%[===================>]     587  --.-KB/s    in 0s      

2021-05-13 02:30:37 (52.8 MB/s) - ‘deepfake_face_points_detection.py’ saved [587/587]



In [6]:
!wget https://github.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/raw/master/Chapter04/deepfake/deepfake_models/shape_predictor_68_face_landmarks.dat

--2021-05-13 02:34:16--  https://github.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/raw/master/Chapter04/deepfake/deepfake_models/shape_predictor_68_face_landmarks.dat
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/master/Chapter04/deepfake/deepfake_models/shape_predictor_68_face_landmarks.dat [following]
--2021-05-13 02:34:16--  https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-for-Cybersecurity-Cookbook/master/Chapter04/deepfake/deepfake_models/shape_predictor_68_face_landmarks.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awa

In [12]:
!mkdir deepfake_models

In [13]:

!mv ./shape_predictor_68_face_landmarks.dat deepfake_models/shape_predictor_68_face_landmarks.dat

Lets start with code.

In [14]:
import os
import cv2
import numpy as np

from deepfake_face_detection import select_face
from deepfake_face_swap import (
    warp_image_2d,
    warp_image_3d,
    mask_from_points,
    apply_mask,
    correct_colours,
    transformation_from_points,
    ProcessFace,
)

Specify the image containing the face we wish to use in content_image and the
image where we want the face to be transferred to in target_image. Finally,
specify where you'd like the result created

In [15]:
!wget -O elon.jpg https://upload.wikimedia.org/wikipedia/commons/8/85/Elon_Musk_Royal_Society_%28crop1%29.jpg
!wget -O witcher.jpg https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/henry-cavill-the-witcher-season-2-1601904500.jpg

--2021-05-13 02:42:01--  https://upload.wikimedia.org/wikipedia/commons/8/85/Elon_Musk_Royal_Society_%28crop1%29.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 171941 (168K) [image/jpeg]
Saving to: ‘elon.jpg’

elon.jpg            100%[===================>] 167.91K   587KB/s    in 0.3s    

2021-05-13 02:42:02 (587 KB/s) - ‘elon.jpg’ saved [171941/171941]

--2021-05-13 02:42:02--  https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/henry-cavill-the-witcher-season-2-1601904500.jpg
Resolving hips.hearstapps.com (hips.hearstapps.com)... 151.101.200.155
Connecting to hips.hearstapps.com (hips.hearstapps.com)|151.101.200.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102744 (100K) [image/jpeg]
Saving to: ‘witcher.jpg’

witcher.jpg         100%[==============

### input images

![](elon.jpg)
![](./witcher.jpg)

In [18]:
content_image = "elon.jpg"
target_image = "witcher.jpg"
result_image_path = "musk_witcher.jpg"

Lets read theimages into opencv and then extract the source and destination faces

In [19]:
content_img = cv2.imread(content_image)
destination_img = cv2.imread(target_image)
content_img_points, content_img_shape, content_img_face = select_face(content_img)

destination_img_points, destination_img_shape, destination_img_face = select_face(destination_img)

compute a transformed version of source face

In [20]:
result_image = ProcessFace(content_img_points, content_img_face, destination_img_points, destination_img_face)

Draw the transformed face into the destination imagea nd write file to disk



In [21]:
x, y, w, h = destination_img_shape
destination_img_copy = destination_img.copy()
destination_img_copy[y : y + h, x: x + w] = result_image
result_image = destination_img_copy
cv2.imwrite(result_image_path, result_image)

True

### The result?

![](musk_witcher.jpg)

err... okay.